In [138]:
import pandas as pd
import numpy as np
import datetime as dt
from string import digits

In [146]:
iso_df = pd.read_csv('../../data/raw/iso_3digit_alpha_country_codes.csv', low_memory=False)
airsavvi = pd.read_csv('../../data/tidy/airsavvi_list.csv')
df = pd.read_csv('../../data/raw/govt_measures.csv')

In [147]:
iso_exclude = ['ALB', 'ISL', 'MAC', 'HKG', 'ROU', 'TWN'] #partially missing data

def series_lower(s):
    return s.str.lower().str

def country_matching(substring):
    remove_digits = str.maketrans('', '', digits)
    substring_lower  = substring.lower()
    iso_lower = series_lower(iso_df['Definition'])
    if substring == 'United States' or substring == 'US' or substring == 'United States of America':
        return 'USA'
    elif substring == 'Russian Federation':
        return 'RUS'
    elif substring == 'India':
        return 'IND'
    elif substring == 'Turkey (Ottoman Empire)':
        return 'TUR'
    elif substring == 'Taiwan*':
        return 'TWN'
    elif substring == 'Republic of Korea' or substring == 'Korea Republic of':
        return 'KOR'
    elif substring == 'Viet Nam':
        return 'VNM'
    elif substring == 'Czech Republic' or substring == 'Czechia':
        return 'CZE'
    elif substring == 'German Federal Republic':
        return 'DEU'
    elif substring == 'Italy/Sardinia':
        return 'ITA'
    elif substring == 'Cambodia (Kampuchea)':
        return 'KHM'
    elif iso_lower.contains(substring_lower).any():
        return iso_df['Code_Value'].loc[iso_lower.find(substring_lower) != -1].to_string().translate(remove_digits).strip()
    else:
        return "N/A"

In [148]:
df['ISO'] = df['COUNTRY'].apply(lambda x: country_matching(x))
airsavvi['iso'] = airsavvi['country'].apply(lambda x: country_matching(x))

df = df.loc[df['ISO'].isin(airsavvi['iso'])]
df = df[df.ISO.apply(lambda x: x not in iso_exclude)]

df = df[['ISO','CATEGORY','ENTRY_DATE']]
df.ENTRY_DATE = pd.to_datetime(df.ENTRY_DATE)

df.head()
#df.CATEGORY.str.replace('Public health measures', 'PH', regex=False)
# PH = public health
# SE = govt and socio-economic
# SD = social distancing
# MR = movement restrictions
# LD = lockdown
# HE = humanitarian exemption

,ISO,CATEGORY,ENTRY_DATE
500,ARG,Movement restrictions,2020-03-14
501,ARG,Movement restrictions,2020-03-14
502,ARG,Public health measures,2020-03-14
503,ARG,Social distancing,2020-03-20
504,ARG,Movement restrictions,2020-03-20


In [149]:
start_date = dt.datetime(2020,2,15)
date_range = pd.date_range(start_date, periods=199, freq='D')

#Set MultiIndex name
idx = pd.MultiIndex.from_product([df.ISO.unique(), df.CATEGORY.unique()])
idx.set_names(['iso', 'category'], inplace=True)

idf = pd.DataFrame(
    data=0,
    index=idx, #later, maybe: ['PH','SE','MR','LD','HE'] instead of actual category names
    columns=date_range
)

#Subset of countries with airsavvi flight data from Feb 15 to Aug 31
idf = idf.loc[idf.index.get_level_values('iso').isin(airsavvi['iso'])] #get_level_values for accessing MultiIndex

idf.head()

2020-02-15  2020-02-16  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-17  2020-02-18  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-19  2020-02-20  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-21  2020-02-22  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-23  2020-02-24  ...  \
iso category                                                        ...   
ARG Movement restrictions                            0           0  ...   
    Public health measures                           0           0  ...   
    Social distancing                                0           0  ...   
    Governance and socio-economic measures           0           0  ...   
    Lockdown                                         0           0  ...   

                                            2020-08-22  2020-08-23  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-24  2020-08-25  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-26  2020-08-27  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-28  2020-08-29  \
iso categor

In [150]:
for i in df.index:
    mdate = df.loc[i].ENTRY_DATE
    if (mdate > dt.datetime(2020,2,14)) & (mdate <= dt.datetime(2020, 9, 1)):
        idf.loc[(df.loc[i].ISO, df.loc[i].CATEGORY),df.loc[i].ENTRY_DATE] = 1

In [152]:
idf.loc[('ARG', 'Public health measures'),'2020-03-14']

1

In [153]:
#Print output in tidy
idf.to_csv('../../data/tidy/govt_interventions.csv')
idf.head()

2020-02-15  2020-02-16  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-17  2020-02-18  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-19  2020-02-20  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-21  2020-02-22  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-23  2020-02-24  ...  \
iso category                                                        ...   
ARG Movement restrictions                            0           0  ...   
    Public health measures                           0           0  ...   
    Social distancing                                0           0  ...   
    Governance and socio-economic measures           0           0  ...   
    Lockdown                                         0           0  ...   

                                            2020-08-22  2020-08-23  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-24  2020-08-25  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-26  2020-08-27  \
iso category                                                         
ARG Movement restrictions                            0           0   
    Public health measures                           0           0   
    Social distancing                                0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-28  2020-08-29  \
iso categor

In [154]:
# Examples
idf.loc[idf.index.get_level_values('category')=='Movement restrictions']

,,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,...,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31
iso,category,,,,,,,,,,,,,,,,,,,,,
ARG,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AUS,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AUT,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BEL,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BRA,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
BGR,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
KHM,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CAN,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHL,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
